![image](competi.png)

#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

In [38]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor


In [73]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("hospital_train.csv")
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
1,159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
2,309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
3,279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
4,147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,237869,12,a,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,More than 100 Days
99996,254763,28,b,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,51-60
99997,69788,6,a,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,31-40
99998,204442,32,f,9,Y,2,gynecology,S,B,4.0,113798,15.0,Trauma,Moderate,3,41-50,5092.0,11-20


In [74]:
df.drop(["1","2"],axis=1,inplace=True)

In [75]:
df.groupby("17").count()

,0,3,4,5,6,7,8,9,10,11,12,13,14,15,16
17,,,,,,,,,,,,,,,
0-10,7427,7427,7427,7427,7427,7427,7427,7426,7427,7306,7427,7427,7427,7427,7427
11-20,24476,24476,24476,24476,24476,24476,24476,24469,24476,24150,24476,24476,24476,24476,24476
21-30,27452,27452,27452,27452,27452,27452,27452,27444,27452,27065,27452,27452,27452,27452,27452
31-40,17311,17311,17311,17311,17311,17311,17311,17303,17311,17050,17311,17311,17311,17311,17311
41-50,3668,3668,3668,3668,3668,3668,3668,3665,3668,3602,3668,3668,3668,3668,3668
51-60,10984,10984,10984,10984,10984,10984,10984,10980,10984,10817,10984,10984,10984,10984,10984
61-70,857,857,857,857,857,857,857,857,857,842,857,857,857,857,857
71-80,3297,3297,3297,3297,3297,3297,3297,3295,3297,3245,3297,3297,3297,3297,3297
81-90,1470,1470,1470,1470,1470,1470,1470,1470,1470,1447,1470,1470,1470,1470,1470


In [76]:
encoder = OrdinalEncoder(categories=[["0-10", "11-20", "21-30", "31-40", "41-50","51-60", "61-70","71-80","81-90","91-100","More than 100 Days"]])
encoder.fit(df[["17"]])
df["17"] = encoder.transform(df[["17"]])

In [77]:
df

,0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,161528,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0.0
1,159472,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,2.0
2,309765,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,1.0
3,279614,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,5.0
4,147791,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,237869,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,10.0
99996,254763,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,5.0
99997,69788,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,3.0
99998,204442,9,Y,2,gynecology,S,B,4.0,113798,15.0,Trauma,Moderate,3,41-50,5092.0,1.0


In [78]:
df.corr()

,0,3,5,9,10,11,14,16,17
0,1.000000,-0.012569,0.044390,0.012169,-0.005268,0.065421,-0.000304,-0.043202,-0.055761
3,-0.012569,1.000000,-0.043746,-0.048573,-0.004483,-0.027284,0.021327,-0.033820,0.008060
5,0.044390,-0.043746,1.000000,-0.118375,0.005043,-0.010412,0.091744,-0.141999,-0.122707
9,0.012169,-0.048573,-0.118375,1.000000,0.005770,-0.010966,0.093038,0.071130,0.029388
10,-0.005268,-0.004483,0.005043,0.005770,1.000000,0.006760,0.007707,-0.001891,0.002740
11,0.065421,-0.027284,-0.010412,-0.010966,0.006760,1.000000,-0.008242,0.025461,-0.002498
14,-0.000304,0.021327,0.091744,0.093038,0.007707,-0.008242,1.000000,-0.152946,0.543491
16,-0.043202,-0.033820,-0.141999,0.071130,-0.001891,0.025461,-0.152946,1.000000,-0.056711
17,-0.055761,0.008060,-0.122707,0.029388,0.002740,-0.002498,0.543491,-0.056711,1.000000


In [79]:

X = np.array(df[["3","5","10","14","16"]])
y = np.array(df["17"])
X.shape


(100000, 5)

In [80]:
y.shape

(100000,)

In [81]:
# el test_size 0.20 y el random_state puede cambiar
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=7)

In [82]:
train_X

array([[6.0000e+00, 3.0000e+00, 2.2111e+04, 3.0000e+00, 4.6250e+03],
       [6.0000e+00, 4.0000e+00, 9.9838e+04, 2.0000e+00, 6.2930e+03],
       [5.0000e+00, 5.0000e+00, 2.2470e+03, 3.0000e+00, 6.0390e+03],
       ...,
       [9.0000e+00, 2.0000e+00, 5.2707e+04, 2.0000e+00, 4.5440e+03],
       [5.0000e+00, 2.0000e+00, 4.7930e+04, 1.4000e+01, 2.9360e+03],
       [1.1000e+01, 4.0000e+00, 4.9829e+04, 3.0000e+00, 5.2080e+03]])

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  int64  
 1   3       100000 non-null  int64  
 2   4       100000 non-null  object 
 3   5       100000 non-null  int64  
 4   6       100000 non-null  object 
 5   7       100000 non-null  object 
 6   8       100000 non-null  object 
 7   9       99967 non-null   float64
 8   10      100000 non-null  int64  
 9   11      98517 non-null   float64
 10  12      100000 non-null  object 
 11  13      100000 non-null  object 
 12  14      100000 non-null  int64  
 13  15      100000 non-null  object 
 14  16      100000 non-null  float64
 15  17      100000 non-null  float64
dtypes: float64(4), int64(5), object(7)
memory usage: 12.2+ MB


In [84]:
rf = RandomForestRegressor(n_estimators=500, random_state=7,n_jobs=5)
rf.fit(train_X, train_y)


RandomForestRegressor(n_estimators=500, n_jobs=5, random_state=7)

In [85]:
predictions = rf.predict(test_X)

In [86]:
errors = abs(predictions - test_y)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 1.32 degrees.


In [87]:
np.sqrt(mean_squared_error(test_y, predictions))

1.790203394005617

-----------------------------------------------------------------

In [88]:
X_pred = pd.read_csv("hospital_test.csv")
X_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
1,208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
2,305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
3,266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
4,13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0


In [89]:
X_pred.drop(["1","2"],axis=1,inplace=True)

In [90]:
X_pred = np.array(X_pred[["3","5","10","14","16"]])


In [91]:
X.shape

(100000, 5)

In [92]:
X_pred.shape

(133744, 5)

In [93]:
predictions_submit = rf.predict(X_pred)
predictions_submit

array([1.592, 1.918, 2.04 , ..., 1.936, 2.24 , 2.016])

In [94]:
X_pred

array([[7.00000e+00, 4.00000e+00, 5.92780e+04, 2.00000e+00, 4.77800e+03],
       [5.00000e+00, 3.00000e+00, 1.02253e+05, 3.00000e+00, 5.73400e+03],
       [1.00000e+00, 4.00000e+00, 5.82800e+03, 3.00000e+00, 5.06400e+03],
       ...,
       [3.00000e+00, 3.00000e+00, 1.12921e+05, 3.00000e+00, 5.32600e+03],
       [1.10000e+01, 4.00000e+00, 5.85000e+02, 2.00000e+00, 7.07200e+03],
       [6.00000e+00, 4.00000e+00, 1.19482e+05, 4.00000e+00, 5.22100e+03]])

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [95]:
sample = pd.read_csv("sample_submission.csv")

In [96]:
sample.head()

,id,days
0,314114,11-20
1,208989,31-40
2,305872,81-90
3,266099,21-30
4,13228,31-40


In [97]:
sample.shape

(133744, 2)

In [98]:
submission = pd.DataFrame({"id": range(len(predictions_submit)), "days": predictions_submit})

In [72]:
submission.head()

,id,days
0,0,1.592
1,1,1.918
2,2,2.040
3,3,1.790
4,4,2.376


In [70]:
submission.shape

(133744, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [75]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [76]:
help(chequeator)

Help on function chequeator in module __main__:

chequeator(df_to_submit)
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.



In [103]:
chequeator(submission)

You're ready to submit!
